In [2]:
import csv, re
f = open('ExpertInterview.csv')
reader = csv.reader(f)
codes = list(reader)

def addToCodeBook(codes):
    all_codes.update(codes)

def cleanCodes(rawCodes):
    rawCodes = re.split(' |,|\n', rawCodes)
    codes = [code.lower().strip() for code in rawCodes if code]
    addToCodeBook(codes)
    return codes

In [3]:
quoteDict = {}
all_codes = set({})
for index, code in enumerate(codes[1:]): 
    if not (code[1] == '' and code[2] == ''):
        quote = {}
        quote['quote'] = code[0]
        quote['chris'] = cleanCodes(code[1])
        quote['alina'] = cleanCodes(code[2])
        quoteDict[index] = quote

In [4]:
all_codes

{'#active',
 '#advantage',
 '#affordance',
 '#agency',
 '#attention',
 '#audienceexpectation',
 '#audienceperforming',
 '#authorcreatorgoaltension',
 '#authoringgoal',
 '#companygoals',
 '#connection',
 '#constraints',
 '#context',
 '#contractofcare',
 '#control',
 '#cost',
 '#creatingmagic',
 '#cues',
 '#culture',
 '#design',
 '#designingfordisengagement',
 '#digital',
 '#disadvantage',
 '#discoverymoment',
 '#emotion',
 '#engagement',
 '#experiencespace',
 '#gesamkunstwerk',
 '#goals',
 '#immersion',
 '#influenceperformers',
 '#inspiration',
 '#interaction',
 '#interactiondisadvantages',
 '#interactionengineering',
 '#marketing',
 '#meaningful',
 '#moodengineering',
 '#morethanspectrum',
 '#mr',
 '#multisensory',
 '#narrative',
 '#narrativecomplexity',
 '#narrativegoal',
 '#novelty',
 '#outcomes',
 '#personalization',
 '#physical',
 '#physicality',
 '#physicallimitation',
 '#practicalchallenges',
 '#presence',
 '#process',
 '#publicprivate',
 '#sensory',
 '#social',
 '#socialengineer

In [5]:
# quoteDict

In [6]:
# Preparing codes for NLTK
from collections import defaultdict

codeDict = defaultdict(dict)
for code in all_codes: 
    for ix, quote in quoteDict.items(): 
        quotes = {}
        quotes['alina'] = 1 if code in quote['alina'] else 0
        quotes['chris'] = 1 if code in quote['chris'] else 0 
        quotes['quote'] = quote['quote']
        codeDict[code][ix] = quotes

In [7]:
# codeDict["#social"]

In [8]:
# Provide a single tag, or multiple tags. 
raters = ['alina', 'chris']
raterFlag = [False] * len(raters)

def getAgreementCodes(tags):
    data = []
    for rater in raters: 
        for itemlabel, quote in quoteDict.items():
            if len(tags) > 1 and all([tag in quote[rater] for tag in tags]):
                data.append([rater, itemlabel, "#Both"])
            if all([tag not in quote[rater] for tag in tags]):
                data.append([rater, itemlabel, "#None"])
            else: 
                for tag in tags:
                    if tag in quote[rater]:
                        data.append([rater, itemlabel, tag])
    return data

In [9]:
# Sets of labels not needed for this dataset. 
# labels = [
#     ("#LackingSupport", "#ReceivingSupport"),
#     ("#Believable", "#NotBelievable"),
#     ("#MiddleGround", "#Disagreement"),
#     ("#StandingGround", "#ChangedOpinion"),
#     ("#Expected", "#Unexpected"),
#     ("#DecreasingCertainty", "#IncreasingCertainty"),
#     ("#UncertainMindsChange", "#ChangedDespiteCertainty"),
#     ("#InfluenceGroup",),
#     ("#InfluenceIndividual",),
#     ("#InfluenceArticle",),
#     ("#ReinforcedViews",),
#     ("#NPCMentionedUnprompted",),
#     ("#UsedPoliticalAffiliationSteriotype",),
#     ("#IdentifyingSimilarGroups",),
#     ("#SimilarViewsConverge",),
#     ("#OpinionAttitudeDifference",),
#     ("#CertaintyConvinces",),
#     ("#PoliticalIdentity",),
#     ("#NotEnoughInfo",),
#     ("#EmotionalResponse",),
#     ("#InferFactsFrom",),
#     ("#CausalInference",),
#     ("#Meta",),
#     ("#GroupsStandGround",),
#     ("#NoAnswer",),
#     ("#Polarization",),
#     ("#Infer",),
#     ("#AgreeWithMajority",),
#     ("#ExtremeChangesInOpinion",),
#     ("#OpinionAttitudeAlignment",),
#     ("#LimitedKnowledge",),
#     ("#OvertonWindow",)
# ]
all_codes = [(label,) for label in all_codes]
all_codes

[('#tech',),
 ('#space',),
 ('#connection',),
 ('#cues',),
 ('#marketing',),
 ('#agency',),
 ('#inspiration',),
 ('#videogame',),
 ('#socialengineering',),
 ('#physical',),
 ('#cost',),
 ('#morethanspectrum',),
 ('#designingfordisengagement',),
 ('#theme',),
 ('#context',),
 ('#active',),
 ('#companygoals',),
 ('#experiencespace',),
 ('#mr',),
 ('#attention',),
 ('#novelty',),
 ('#engagement',),
 ('#advantage',),
 ('#outcomes',),
 ('#authorcreatorgoaltension',),
 ('#theatre',),
 ('#physicallimitation',),
 ('#values',),
 ('#physicality',),
 ('#interactiondisadvantages',),
 ('#goals',),
 ('#narrative',),
 ('#design',),
 ('#narrativecomplexity',),
 ('#authoringgoal',),
 ('#meaningful',),
 ('#control',),
 ('#publicprivate',),
 ('#affordance',),
 ('#narrativegoal',),
 ('#audienceexpectation',),
 ('#interactionengineering',),
 ('#transmedia',),
 ('#spectrum',),
 ('#culture',),
 ('#digital',),
 ('#audienceperforming',),
 ('#emotion',),
 ('#sensory',),
 ('#process',),
 ('#visualdesign',),
 ('#

In [11]:
from nltk.metrics.agreement import AnnotationTask
from nltk.probability import FreqDist

with open('irr_calculations.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(["Tag", "Kappa", "Fleiss", "Alpha", "Scotts"])

    total_fleiss = []
    total_kappa = []
    total_alpha = []
    total_scotts = []

    for code in all_codes: 
        nltk_code_format = getAgreementCodes(code)
        task = AnnotationTask(data=nltk_code_format)
        label_freqs = FreqDist(x['labels'] for x in task.data)
        print("Codes: ", label_freqs.keys())
        try: 
            kappa = str(task.kappa())
            total_kappa.append(task.kappa())
            fleiss = task.multi_kappa()
            total_fleiss.append(task.multi_kappa())
            alpha = str(task.alpha())
            total_alpha.append(task.alpha())
            scotts = str(task.pi())
            total_scotts.append(task.pi())
            csvwriter.writerow([[str(key) for key in label_freqs.keys()], kappa, fleiss, alpha, scotts])
            print([[str(key) for key in label_freqs.keys()], kappa, fleiss, alpha, scotts])

        except ZeroDivisionError as e: 
            print("Expected Agreement: 1.0 - ZeroDivisionError")

    tot_fleiss = sum(total_fleiss)/len(total_fleiss)
    tot_kappa = sum(total_kappa)/len(total_kappa)
    tot_alpha = sum(total_alpha)/len(total_alpha)
    tot_scotts = sum(total_scotts)/len(total_scotts)
    csvwriter.writerow(["Total", tot_kappa, tot_fleiss, tot_alpha, tot_scotts])
    print(["Total", tot_kappa, tot_fleiss, tot_alpha, tot_scotts])

Codes:  dict_keys(['#None', '#tech'])
[['#None', '#tech'], '0.9544851966416261', 0.9544851966416261, '0.9546961325966851', '0.9544751381215469']
Codes:  dict_keys(['#None', '#space'])
[['#None', '#space'], '0.790224032586558', 0.790224032586558, '0.7908163265306123', '0.789795918367347']
Codes:  dict_keys(['#None', '#connection'])
[['#None', '#connection'], '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#cues'])
[['#None', '#cues'], '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#marketing'])
[['#None', '#marketing'], '0.7897959183673467', 0.7897959183673467, '0.7908163265306123', '0.789795918367347']
Codes:  dict_keys(['#None', '#agency'])
[['#None', '#agency'], '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#inspiration'])
[['#None', '#inspiration'], '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#videogame'])
[['#None', '#videogame'], '1.0', 1.0, '1.0', '1.0']
Codes:  dict_keys(['#None', '#socialengineering'])
[['#None', '#socialengineering'], '0.6622